# Bachelor End Project (BEP) - Els Brouwer - May 2025

## Set up

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import time
import os

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tqdm import tqdm  # progress bar
from collections import Counter

Check sklearn version for default values used

In [2]:
import sklearn
print(sklearn.__version__)

1.6.1


# 2. Non-Linear Data

## 2.1 Generating Non Linear High Dimensional Data Sets 

In [3]:
def generate_nonlinear_data(n_observations, n_informative, informative_ratio, noise_amplitude, random_seed, function_number):
    # Set random seed for reproducibility
    np.random.seed(random_seed)

    # Compute total number of features
    n_features = int(n_informative / informative_ratio)

    # Generate random feature matrix
    X = np.random.randn(n_observations, n_features)

    # Extract only the first 5 informative features for signal generation
    X1, X2, X3, X4, X5 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    # Compute nonlinear signal based on function number
    if function_number == 1:
        # Additive sine and square
        nonlinear_part = np.sin(X1) + X2**2 + np.sin(X3) + X4**2 + np.sin(X5)

    elif function_number == 2:
        # Pairwise interactions with arctan
        nonlinear_part = np.arctan(X1 * X2) + np.arctan(X3 * X4) + np.arctan(X5)

    elif function_number == 3:
        # Compositional nonlinearity with log and sin
        nonlinear_part = np.log(np.abs(np.sin(X1) * X2) + 1) + \
                         np.log(np.abs(np.sin(X3) * X4) + 1) + X5

    elif function_number == 4:
        # Thresholded quadratics (sparse)
        nonlinear_part = np.where(X1 > 1, X1**2, 0) + \
                         np.where(X2 > 1, X2**2, 0) + \
                         np.where(X3 > 1, X3**2, 0) + \
                         np.where(X4 > 1, X4**2, 0) + \
                         np.where(X5 > 1, X5**2, 0)

    elif function_number == 5:
        # Mixed univariate nonlinearities
        nonlinear_part = np.sin(X1) + np.log(np.abs(X2) + 1) + X3**2 + \
                         np.arctan(X4) + np.exp(X5)

    elif function_number == 6:
        # Interaction + composition
        nonlinear_part = (X1 * X2) + np.sqrt(np.abs(X3)) + \
                         np.log2(np.abs(X4) + 1) + np.sin(X5)

    elif function_number == 7:
        # Deep composition
        nonlinear_part = np.sin(np.log(np.abs(X1 * X2) + 1)) + \
                         (X3 * X4) + np.exp(np.sqrt(np.abs(X5)))

    else:
        raise ValueError("function_number must be an integer between 1 and 7")

    # Add Gaussian noise
    y = nonlinear_part + noise_amplitude * np.random.randn(n_observations)

    return X, y, n_features

## 2.2 Penalized Regression with Ridge (Non-Linear)

In [ ]:
# # === Parameters ===
# n_observations_train = 100  # Fixed number of observations for training
# n_observations_test = 100  # Test set of 100 observations
# n_observations = n_observations_test + n_observations_train  # Total observations
# n_informative = 5  # Number of truly informative features
# n_simulations = 50
# n_folds = 5
# alpha_grid = {'ridge__alpha': np.logspace(-4, 4, 30)}
# informative_ratios = [0.01]
# noise_amplitudes = [0.01]
# function_numbers = range(1, 8)  # Nonlinear functions 1 through 7

# # Store combined mean results
# ridge_nonlinear_functions_results = []

# # === Loop over functions ===
# for func_num in function_numbers:
#     print(f"\n=== Running simulations for nonlinear function #{func_num} ===")

#     for noise_amplitude in noise_amplitudes:
#         print(f"Noise amplitude: {noise_amplitude}")

#         for informative_ratio in informative_ratios:
#             print(f"Informative ratio: {informative_ratio}")

#             # Initialize result lists for this setting
#             mse_list = []
#             precision5_list = []
#             precision10_list = []

#             # === Simulations ===
#             for sim in tqdm(range(n_simulations), desc=f"Func {func_num} - Simulations"):
#                 seed = 42 + sim

#                 # === Data generation ===
#                 X, y, n_features = generate_nonlinear_data(
#                     n_observations,
#                     n_informative,
#                     informative_ratio,
#                     noise_amplitude,
#                     seed,
#                     func_num
#                 )

#                 # Split into train and test
#                 X_train, y_train = X[:n_observations_train], y[:n_observations_train]
#                 X_test, y_test = X[n_observations_train:], y[n_observations_train:]

#                 # === Ridge pipeline ===
#                 ridge_pipeline = Pipeline([
#                     ('scaler', StandardScaler()),
#                     ('ridge', Ridge(random_state=42))
#                 ])

#                 grid_search = GridSearchCV(
#                     ridge_pipeline,
#                     param_grid=alpha_grid,
#                     cv=n_folds,
#                     scoring='neg_mean_squared_error',
#                     n_jobs=-1
#                 )

#                 # === Fit model ===
#                 grid_search.fit(X_train, y_train)
#                 best_model = grid_search.best_estimator_

#                 # === Evaluate on test set ===
#                 y_pred = best_model.predict(X_test)
#                 mse = mean_squared_error(y_test, y_pred)
#                 mse_list.append(mse)

#                 # === Precision@k ===
#                 coef = best_model.named_steps['ridge'].coef_
#                 coef_magnitudes = np.abs(coef)

#                 top_5_indices = np.argsort(coef_magnitudes)[-5:][::-1]
#                 top_10_indices = np.argsort(coef_magnitudes)[-10:][::-1]

#                 correct_top_5 = np.sum(top_5_indices < n_informative)
#                 correct_top_10 = np.sum(top_10_indices < n_informative)

#                 precision5 = correct_top_5 / 5
#                 precision10 = correct_top_10 / 5

#                 precision5_list.append(precision5)
#                 precision10_list.append(precision10)

#             # === Aggregate results ===
#             mean_mse = np.mean(mse_list)
#             std_mse = np.std(mse_list)
#             mean_precision5 = np.mean(precision5_list)
#             std_precision5 = np.std(precision5_list)
#             mean_precision10 = np.mean(precision10_list)
#             std_precision10 = np.std(precision10_list)

#             ridge_nonlinear_functions_results.append({
#                 'function_number': func_num,
#                 'noise': noise_amplitude,
#                 'informative_ratio': informative_ratio,
#                 'n_features': n_features,
#                 'mean_mse': mean_mse,
#                 'std_mse': std_mse,
#                 'mean_precision5': mean_precision5,
#                 'std_precision5': std_precision5,
#                 'mean_precision10': mean_precision10,
#                 'std_precision10': std_precision10
#             })

# # === Save results ===
# df_ridge_results = pd.DataFrame(ridge_nonlinear_functions_results)
# df_ridge_results.to_csv('Ridge_nonlinear_functions_mean_MSE.csv', index=False)
# print("Saved Ridge nonlinear functions mean MSE results to 'Ridge_nonlinear_functions_mean_MSE.csv'")


: 

## 2.3 Penalized Regression with Lasso (Non-Linear)

In [ ]:
# # === Parameters ===
# n_observations_train = 100  # Fixed number of observations for training
# n_observations_test = 100  # Test set of 100 observations
# n_observations = n_observations_test + n_observations_train  # Total observations
# n_informative = 5  # Number of truly informative features
# n_simulations = 50
# n_folds = 5
# alpha_grid = {'lasso__alpha': np.logspace(-4, 4, 30)}
# informative_ratios = [0.01]
# noise_amplitudes = [0.01]
# function_numbers = range(1, 8)  # Nonlinear functions 1 through 7

# # Store combined mean results
# lasso_nonlinear_functions_results = []

# # === Loop over functions ===
# for func_num in function_numbers:
#     print(f"\n=== Running simulations for nonlinear function #{func_num} ===")

#     for noise_amplitude in noise_amplitudes:
#         print(f"Noise amplitude: {noise_amplitude}")

#         for informative_ratio in informative_ratios:
#             print(f"Informative ratio: {informative_ratio}")

#             # Initialize result lists for this setting
#             mse_list = []
#             precision5_list = []
#             precision10_list = []

#             # === Simulations ===
#             for sim in tqdm(range(n_simulations), desc=f"Func {func_num} - Simulations"):
#                 seed = 42 + sim

#                 # === Data generation ===
#                 X, y, n_features = generate_nonlinear_data(
#                     n_observations,
#                     n_informative,
#                     informative_ratio,
#                     noise_amplitude,
#                     seed,
#                     func_num
#                 )

#                 # Split into train and test
#                 X_train, y_train = X[:n_observations_train], y[:n_observations_train]
#                 X_test, y_test = X[n_observations_train:], y[n_observations_train:]

#                 # === Lasso pipeline ===
#                 lasso_pipeline = Pipeline([
#                     ('scaler', StandardScaler()),
#                     ('lasso', Lasso(max_iter=10000, random_state=42))
#                 ])

#                 grid_search = GridSearchCV(
#                     lasso_pipeline,
#                     param_grid=alpha_grid,
#                     cv=n_folds,
#                     scoring='neg_mean_squared_error',
#                     n_jobs=-1
#                 )

#                 # === Fit model ===
#                 grid_search.fit(X_train, y_train)
#                 best_model = grid_search.best_estimator_

#                 # === Evaluate on test set ===
#                 y_pred = best_model.predict(X_test)
#                 mse = mean_squared_error(y_test, y_pred)
#                 mse_list.append(mse)

#                 # === Precision@k ===
#                 coef = best_model.named_steps['lasso'].coef_
#                 coef_magnitudes = np.abs(coef)

#                 top_5_indices = np.argsort(coef_magnitudes)[-5:][::-1]
#                 top_10_indices = np.argsort(coef_magnitudes)[-10:][::-1]

#                 correct_top_5 = np.sum(top_5_indices < n_informative)
#                 correct_top_10 = np.sum(top_10_indices < n_informative)

#                 precision5 = correct_top_5 / 5
#                 precision10 = correct_top_10 / 5

#                 precision5_list.append(precision5)
#                 precision10_list.append(precision10)

#             # === Aggregate results ===
#             mean_mse = np.mean(mse_list)
#             std_mse = np.std(mse_list)
#             mean_precision5 = np.mean(precision5_list)
#             std_precision5 = np.std(precision5_list)
#             mean_precision10 = np.mean(precision10_list)
#             std_precision10 = np.std(precision10_list)

#             lasso_nonlinear_functions_results.append({
#                 'function_number': func_num,
#                 'noise': noise_amplitude,
#                 'informative_ratio': informative_ratio,
#                 'n_features': n_features,
#                 'mean_mse': mean_mse,
#                 'std_mse': std_mse,
#                 'mean_precision5': mean_precision5,
#                 'std_precision5': std_precision5,
#                 'mean_precision10': mean_precision10,
#                 'std_precision10': std_precision10
#             })

# # === Save results ===
# df_lasso_results = pd.DataFrame(lasso_nonlinear_functions_results)
# df_lasso_results.to_csv('Lasso_nonlinear_functions_mean_MSE.csv', index=False)
# print("Saved Lasso nonlinear functions mean MSE results to 'Lasso_nonlinear_functions_mean_MSE.csv'")


: 

## 2.4 Random Forest (Non-Linear)

In [ ]:
# # === Parameters ===
# n_observations_train = 100  # Fixed number of observations for training
# n_observations_test = 100   # Test set of 100 observations
# n_observations = n_observations_train + n_observations_test  # Total observations
# n_informative = 5           # Number of truly informative features
# n_simulations = 50
# n_folds = 5
# informative_ratios = [0.01]
# noise_amplitudes = [0.01]
# function_numbers = range(1, 8)  # Nonlinear functions 1 through 7

# # Store combined mean results
# rf_nonlinear_functions_results = []

# for func_num in function_numbers:
#     print(f"\n=== Running simulations for nonlinear function #{func_num} ===")

#     for noise_amplitude in noise_amplitudes:
#         print(f"Noise amplitude: {noise_amplitude}")

#         for informative_ratio in informative_ratios:
#             print(f"Informative ratio: {informative_ratio}")

#             mse_list = []
#             precision5_list = []
#             precision10_list = []

#             for sim in tqdm(range(n_simulations), desc=f"Func {func_num} - Simulations"):
#                 seed = 42 + sim

#                 # === Generate data using specified nonlinear function ===
#                 X, y, n_features = generate_nonlinear_data(
#                     n_observations,
#                     n_informative,
#                     informative_ratio,
#                     noise_amplitude,
#                     seed,
#                     func_num
#                 )

#                 # Split explicitly into 100 train and 100 test
#                 X_train, y_train = X[:n_observations_train], y[:n_observations_train]
#                 X_test, y_test = X[n_observations_train:], y[n_observations_train:]

#                 # === Random Forest model ===
#                 rf = RandomForestRegressor(random_state=42, n_jobs=-1)

#                 # Parameter grid for hyperparameter tuning
#                 rf_param_grid = {
#                     'n_estimators': [100, 200, 300, 400, 500],
#                     'max_depth': [None, 10, 20],
#                     'min_samples_split': [2, 5],
#                     'min_samples_leaf': [1, 2, 4],
#                     'max_features': ['sqrt', 'log2', 1, max(1, n_features // 3)]
#                 }

#                 # Grid search with cross-validation
#                 grid_search = GridSearchCV(
#                     rf,
#                     param_grid=rf_param_grid,
#                     cv=n_folds,
#                     scoring='neg_mean_squared_error',
#                     n_jobs=-1
#                 )

#                 # Fit model
#                 grid_search.fit(X_train, y_train)
#                 best_model = grid_search.best_estimator_

#                 # Predict and evaluate
#                 y_pred = best_model.predict(X_test)
#                 mse = mean_squared_error(y_test, y_pred)
#                 mse_list.append(mse)

#                 # === Compute precision@5 and precision@10 using feature importances ===
#                 importances = best_model.feature_importances_
#                 top_5_indices = np.argsort(importances)[-5:][::-1]
#                 top_10_indices = np.argsort(importances)[-10:][::-1]

#                 correct_top_5 = np.sum(top_5_indices < n_informative)
#                 correct_top_10 = np.sum(top_10_indices < n_informative)

#                 precision5 = correct_top_5 / 5
#                 precision10 = correct_top_10 / 10 # Mistake! Later corrected

#                 precision5_list.append(precision5)
#                 precision10_list.append(precision10)

#             # Aggregate metrics
#             mean_mse = np.mean(mse_list)
#             std_mse = np.std(mse_list)
#             mean_precision5 = np.mean(precision5_list)
#             std_precision5 = np.std(precision5_list)
#             mean_precision10 = np.mean(precision10_list)
#             std_precision10 = np.std(precision10_list)

#             rf_nonlinear_functions_results.append({
#                 'function_number': func_num,
#                 'noise': noise_amplitude,
#                 'informative_ratio': informative_ratio,
#                 'n_features': n_features,
#                 'mean_mse': mean_mse,
#                 'std_mse': std_mse,
#                 'mean_precision5': mean_precision5,
#                 'std_precision5': std_precision5,
#                 'mean_precision10': mean_precision10,
#                 'std_precision10': std_precision10
#             })

# # Save results to CSV
# df_rf_results = pd.DataFrame(rf_nonlinear_functions_results)
# df_rf_results.to_csv('RF_nonlinear_functions_mean_MSE.csv', index=False)
# print("Saved Random Forest nonlinear functions mean MSE results to 'RF_nonlinear_functions_mean_MSE.csv'")


: 

## 2.5 Results Non-Linear Data

In [12]:
# === Load results ===
df_ridge = pd.read_csv('Ridge_nonlinear_functions_mean_MSE.csv')
df_lasso = pd.read_csv('Lasso_nonlinear_functions_mean_MSE.csv')
df_rf = pd.read_csv('RF_nonlinear_functions_mean_MSE.csv')

# === Rename columns to avoid collision ===
df_ridge = df_ridge.rename(columns={'mean_mse': 'ridge_mse'})
df_lasso = df_lasso.rename(columns={'mean_mse': 'lasso_mse'})
df_rf = df_rf.rename(columns={'mean_mse': 'rf_mse'})
# === Rename columns to avoid collision ===
df_rf = df_rf.rename(columns={
    'mean_precision5': 'rf_precision5',
    'mean_precision10': 'rf_precision10'
})
df_lasso = df_lasso.rename(columns={
    'mean_precision5': 'lasso_precision5',
    'mean_precision10': 'lasso_precision10'
})
df_ridge = df_ridge.rename(columns={
    'mean_precision5': 'ridge_precision5',
    'mean_precision10': 'ridge_precision10'
})


# === Merge on common keys ===
merge_keys = ['function_number', 'noise', 'informative_ratio', 'n_features']

# Merge ridge metrics (MSE + precision)
df_merged = df_rf.merge(
    df_ridge[merge_keys + ['ridge_mse', 'ridge_precision5', 'ridge_precision10']], 
    on=merge_keys
)

# Merge lasso metrics (MSE + precision)
df_merged = df_merged.merge(
    df_lasso[merge_keys + ['lasso_mse', 'lasso_precision5', 'lasso_precision10']], 
    on=merge_keys
)

df_merged["rf_precision10_corrected"] = df_merged["rf_precision10"]*2

In [13]:
print(df_merged)

   function_number  noise  informative_ratio  n_features    rf_mse   std_mse  \
0                1   0.01               0.01         500  4.452820  1.215673   
1                2   0.01               0.01         500  0.842715  0.125393   
2                3   0.01               0.01         500  0.468622  0.096619   
3                4   0.01               0.01         500  4.667571  1.598016   
4                5   0.01               0.01         500  4.772339  1.681232   
5                6   0.01               0.01         500  1.604780  0.356657   
6                7   0.01               0.01         500  1.762679  0.371288   

   rf_precision5  std_precision5  rf_precision10  std_precision10  ridge_mse  \
0          0.604        0.172000           0.380         0.093808   5.241822   
1          0.224        0.064992           0.128         0.049153   1.108070   
2          0.228        0.069397           0.130         0.053852   1.020078   
3          0.760        0.132665       

In [7]:
test_df_merged = df_merged.copy()

# Absolute differences
test_df_merged["difference_Ridge"] = test_df_merged["ridge_mse"] - test_df_merged["rf_mse"]
test_df_merged["difference_Lasso"] = test_df_merged["lasso_mse"] - test_df_merged["rf_mse"]
test_df_merged["difference_mean"] = (test_df_merged["difference_Ridge"] + test_df_merged["difference_Lasso"]) / 2

# Percentage improvements
test_df_merged["pct_improvement_Ridge"] = 100 * (test_df_merged["ridge_mse"] - test_df_merged["rf_mse"]) / test_df_merged["ridge_mse"]
test_df_merged["pct_improvement_Lasso"] = 100 * (test_df_merged["lasso_mse"] - test_df_merged["rf_mse"]) / test_df_merged["lasso_mse"]
test_df_merged["pct_improvement_mean"] = (test_df_merged["pct_improvement_Ridge"] + test_df_merged["pct_improvement_Lasso"]) / 2
